## Adjust to semantic model table & column names
This notebook uses Semantic Link Labs to prettify and update table and column names in bulk

#### How to rename columns in compatibility level 1550 (PBIX mode)

**At this level, you cannot programmatically rename columns via TOM in Fabric / Semantic Link.**

The supported approaches are:
- Use Tabular Editor (externally) to rename the columns — this is fully supported in Power BI mode.
- Use the “transform” layer in the dataset itself — i.e., in the Lakehouse / dataflow / ETL, rename the columns before they enter the semantic model.
- Upgrade the dataset to compatibility level ≥ 1567 (Premium/SSAS mode) — then ChangedProperties and programmatic renames work.

In short: at 1550, Semantic Link cannot save column renames back to the model. That’s why all your attempts appear to work in-memory but nothing persists.

_Note: Just setting the workspace to Large Storage Format is not enough._

In [ ]:
%pip install semantic-link-labs
import sempy_labs as labs
import sempy.fabric as fabric
from sempy_labs import tom

StatementMeta(, f83c1cb0-b136-4c91-b7d2-e0ce778d825f, 8, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.7/764.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 81.4 MB/s eta 0:00:00
  Attempting uninstall: azure-core
    Found existing installation: azure-core 1.30.2
    Not uninstalling azure-core at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-1467723f-7ef3-4d50-af01-a97ff22052a5
    Can't uninstall 'azure-core'. No files were found to uninstall.
  Attempting uninstall: semantic-link-sempy
    Found existing installation: semantic-link-sempy 0.10.2
    Not uninstalling semantic-link-sempy at /home/trusted-service-user/cluste

In [ ]:
semanticmodel_name = "AW2020_old_compat_level"
workspace_name = "Semantic Showdown"

StatementMeta(, f83c1cb0-b136-4c91-b7d2-e0ce778d825f, 10, Finished, Available, Finished)

### Helper function to detect and rename
This function helps to rename items 

In [ ]:
def prettify_name(name: str) -> str:
    chars_to_remove = set(" _-.()[]")
    if not name:
        return name

    result = [name[0]]
    for i in range(1, len(name)):
        ch = name[i]
        if ch in chars_to_remove:
            continue
        if ch.isupper() and name[i-1].islower() and result[-1] != " ":
            result.append(" ")
        result.append(ch)
    return "".join(result)

StatementMeta(, f83c1cb0-b136-4c91-b7d2-e0ce778d825f, 11, Finished, Available, Finished)

## Update table and column names
This section uses before defined function to rename tables and columns in bulk. 

In [ ]:
# Create a read/write TOM connection
tw = tom.TOMWrapper(
    dataset = semanticmodel_name,
    workspace = workspace_name,
    readonly = False
)

StatementMeta(, f83c1cb0-b136-4c91-b7d2-e0ce778d825f, 12, Finished, Available, Finished)

In [ ]:
for table in tw.model.Tables:
    for col in table.Columns:
        if "RowNumber" in col.Name:
            continue
        new_name = prettify_name(col.Name)
        if new_name != col.Name:
            print(f"Renaming {table.Name}.{col.Name} -> {new_name}")
            tw.update_column(table.Name, col.Name, new_name)

StatementMeta(, f83c1cb0-b136-4c91-b7d2-e0ce778d825f, 13, Finished, Available, Finished)

Renaming FactInternetSales.SalesOrderNumber -> Sales Order Number
Renaming FactInternetSales.SalesOrderLineNumber -> Sales Order Line Number
Renaming FactInternetSales.CustomerKey -> Customer Key
Renaming FactInternetSales.ProductKey -> Product Key
Renaming FactInternetSales.OrderDateKey -> Order Date Key
Renaming FactInternetSales.DueDateKey -> Due Date Key
Renaming FactInternetSales.ShipDateKey -> Ship Date Key
Renaming FactInternetSales.PromotionKey -> Promotion Key
Renaming FactInternetSales.CurrencyKey -> Currency Key
Renaming FactInternetSales.SalesTerritoryKey -> Sales Territory Key
Renaming FactInternetSales.OrderQuantity -> Order Quantity
Renaming FactInternetSales.UnitPrice -> Unit Price
Renaming FactInternetSales.ExtendedAmount -> Extended Amount
Renaming FactInternetSales.UnitPriceDiscountPct -> Unit Price Discount Pct
Renaming FactInternetSales.DiscountAmount -> Discount Amount
Renaming FactInternetSales.ProductStandardCost -> Product Standard Cost
Renaming FactInternetSal

In [ ]:
# Apply renames by setting the column Name directly
renamed_cols = []

for table in tw.model.Tables:
    for col in table.Columns:
        if "RowNumber" in col.Name:
            continue

        new_name = prettify_name(col.Name)
        if new_name != col.Name:
            print(f"Renaming {table.Name}.{col.Name} -> {new_name}")
            col.Name = new_name                # <-- directly set the TOM object
            tw.add_changed_property(col, "Name")  # <-- mark it as changed
            renamed_cols.append((table.Name, new_name))

# Persist all changes
tw.model.SaveChanges()
fabric.refresh_tom_cache(workspace_name)

# Print only columns that were renamed
print("\nRenamed columns:")
for table_name, col_name in renamed_cols:
    print(f"{table_name}.{col_name}")

StatementMeta(, f83c1cb0-b136-4c91-b7d2-e0ce778d825f, 14, Finished, Available, Finished)

Renaming FactInternetSales.SalesOrderNumber -> Sales Order Number
Renaming FactInternetSales.SalesOrderLineNumber -> Sales Order Line Number
Renaming FactInternetSales.CustomerKey -> Customer Key
Renaming FactInternetSales.ProductKey -> Product Key
Renaming FactInternetSales.OrderDateKey -> Order Date Key
Renaming FactInternetSales.DueDateKey -> Due Date Key
Renaming FactInternetSales.ShipDateKey -> Ship Date Key
Renaming FactInternetSales.PromotionKey -> Promotion Key
Renaming FactInternetSales.CurrencyKey -> Currency Key
Renaming FactInternetSales.SalesTerritoryKey -> Sales Territory Key
Renaming FactInternetSales.OrderQuantity -> Order Quantity
Renaming FactInternetSales.UnitPrice -> Unit Price
Renaming FactInternetSales.ExtendedAmount -> Extended Amount
Renaming FactInternetSales.UnitPriceDiscountPct -> Unit Price Discount Pct
Renaming FactInternetSales.DiscountAmount -> Discount Amount
Renaming FactInternetSales.ProductStandardCost -> Product Standard Cost
Renaming FactInternetSal